In [208]:
import pandas as pd
import glob,os
import re


In [209]:
OMOP_FILES_PATH = 'codelists/OMOP/'

omop_concepts  = pd.read_csv(OMOP_FILES_PATH+'CONCEPT.csv', sep='\t')
omop_relationships  = pd.read_csv(OMOP_FILES_PATH+'RELATIONSHIP.csv', sep='\t')
omop_concept_relationships  = pd.read_csv(OMOP_FILES_PATH+'CONCEPT_RELATIONSHIP.csv', sep='\t')


omop_concepts

/var/folders/jv/r9wkk1k157jgllfhz9lygd0r0000gp/T/ipykernel_22416/1064190685.py:3: DtypeWarning: Columns (6,9) have mixed types. Specify dtype option on import or set low_memory=False.
  omop_concepts  = pd.read_csv(OMOP_FILES_PATH+'CONCEPT.csv', sep='\t')


,concept_id,concept_name,domain_id,vocabulary_id,concept_class_id,standard_concept,concept_code,valid_start_date,valid_end_date,invalid_reason
0,1146945,concept.concept_id,Metadata,CDM,Field,S,CDM1,20141111,20991231,NaN
1,1146954,concept.invalid_reason,Metadata,CDM,Field,S,CDM10,20141111,20991231,NaN
2,1147044,observation_period.observation_period_id,Metadata,CDM,Field,S,CDM100,20141111,20991231,NaN
3,756315,metadata.metadata_type_concept_id,Metadata,CDM,Field,S,CDM1000,20210925,20991231,NaN
4,756316,metadata.name,Metadata,CDM,Field,S,CDM1001,20210925,20991231,NaN
...,...,...,...,...,...,...,...,...,...,...
1473553,33091,IQVIA CO-CONNECT,Metadata,Vocabulary,Vocabulary,NaN,OMOP generated,19700101,20991231,NaN
1473554,33092,IQVIA CO-CONNECT MIABIS,Metadata,Vocabulary,Vocabulary,NaN,OMOP generated,19700101,20991231,NaN
1473555,33093,IQVIA CO-CONNECT TWINS,Metadata,Vocabulary,Vocabulary,NaN,OMOP generated,19700101,20991231,NaN
1473556,33095,NHS Ethnic Category,Metadata,Vocabulary,Vocabulary,NaN,OMOP generated,19700101,20991231,NaN


In [210]:
s='colorectal'
files = sorted( list( glob.glob("codelists/output/query_"+s+"*_snomed.csv") ) )
files


['codelists/output/query_colorectal_0_snomed.csv',
 'codelists/output/query_colorectal_1_snomed.csv',
 'codelists/output/query_colorectal_2_snomed.csv',
 'codelists/output/query_colorectal_3_snomed.csv',
 'codelists/output/query_colorectal_4_snomed.csv',
 'codelists/output/query_colorectal_5_snomed.csv',
 'codelists/output/query_colorectal_6_snomed.csv',
 'codelists/output/query_colorectal_7_snomed.csv']

In [211]:
for f in files:
    name = re.split('/|\.', f)[-2]
    input_snomed_list = pd.read_csv(f)

    input_snomed_list['conceptId'] = input_snomed_list['conceptId'].astype(str)
    omop_concepts['concept_code'] = omop_concepts['concept_code'].astype(str)
    omop_concepts['concept_id'] = omop_concepts['concept_id'].astype(str)

    omop_concepts_filtered = omop_concepts[ omop_concepts['concept_code'].isin(input_snomed_list['conceptId']) ]

    omop_concepts_filtered = omop_concepts_filtered[['concept_id','concept_name','vocabulary_id','concept_code']]
    omop_concepts_filtered

    #omop_relationships_filtered = omop_concept_relationships[ omop_concept_relationships['relationship_id'].isin(['Maps to','Subsumes'])]

    omop_concept_relationships['concept_id_1'] = omop_concept_relationships['concept_id_1'].astype(str)
    omop_concept_relationships['concept_id_2'] = omop_concept_relationships['concept_id_2'].astype(str)

    concept_maps = omop_concepts_filtered.merge( omop_concept_relationships, how='left', left_on='concept_id', right_on='concept_id_1' )
    concept_maps_named = concept_maps.merge(omop_concepts, how='left', left_on='concept_id_2', right_on='concept_id' )

    target_codes_icd10 = concept_maps_named[ concept_maps_named['vocabulary_id_y']=='ICD10PCS' ]
    target_codes_icd10['opcs'] = target_codes_icd10['concept_code_y']
    print( target_codes_icd10['relationship_id'].value_counts() )

    target_codes_icd9 = concept_maps_named[ concept_maps_named['vocabulary_id_y']=='ICD9Proc' ]
    target_codes_icd9['opcs'] = target_codes_icd9['concept_code_y']
    print( target_codes_icd9['relationship_id'].value_counts() )

    target_codes_opcs4 = concept_maps_named[ concept_maps_named['vocabulary_id_y']=='OPCS4' ]
    target_codes_opcs4['opcs'] = target_codes_opcs4['concept_code_y'].str.replace('.','')

    print( target_codes_opcs4['relationship_id'].value_counts() )

    target_codes_icd9[['concept_code_y','concept_name_y','opcs']].drop_duplicates().to_csv('codelists/output/omop/'+name+'_icd9.csv')
    target_codes_icd10[['concept_code_y','concept_name_y','opcs']].drop_duplicates().to_csv('codelists/output/omop/'+name+'_icd10.csv')
    target_codes_opcs4[['concept_code_y','concept_name_y','opcs']].drop_duplicates().to_csv('codelists/output/omop/'+name+'_opcs.csv')


/var/folders/jv/r9wkk1k157jgllfhz9lygd0r0000gp/T/ipykernel_22416/157034202.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  target_codes_icd10['opcs'] = target_codes_icd10['concept_code_y']
/var/folders/jv/r9wkk1k157jgllfhz9lygd0r0000gp/T/ipykernel_22416/157034202.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  target_codes_icd9['opcs'] = target_codes_icd9['concept_code_y']
/var/folders/jv/r9wkk1k157jgllfhz9lygd0r0000gp/T/ipykernel_22416/157034202.py:31: SettingWithCopyWarning: 
A value is trying t

relationship_id
Subsumes    4745
Name: count, dtype: int64
relationship_id
Mapped from    90
Subsumes       65
Name: count, dtype: int64
relationship_id
Mapped from    274
Subsumes        74
Name: count, dtype: int64


/var/folders/jv/r9wkk1k157jgllfhz9lygd0r0000gp/T/ipykernel_22416/157034202.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  target_codes_icd10['opcs'] = target_codes_icd10['concept_code_y']
/var/folders/jv/r9wkk1k157jgllfhz9lygd0r0000gp/T/ipykernel_22416/157034202.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  target_codes_icd9['opcs'] = target_codes_icd9['concept_code_y']
/var/folders/jv/r9wkk1k157jgllfhz9lygd0r0000gp/T/ipykernel_22416/157034202.py:31: SettingWithCopyWarning: 
A value is trying t

relationship_id
Subsumes    4603
Name: count, dtype: int64
relationship_id
Mapped from    65
Subsumes       47
Name: count, dtype: int64
relationship_id
Mapped from    220
Subsumes        63
Name: count, dtype: int64


/var/folders/jv/r9wkk1k157jgllfhz9lygd0r0000gp/T/ipykernel_22416/157034202.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  target_codes_icd10['opcs'] = target_codes_icd10['concept_code_y']
/var/folders/jv/r9wkk1k157jgllfhz9lygd0r0000gp/T/ipykernel_22416/157034202.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  target_codes_icd9['opcs'] = target_codes_icd9['concept_code_y']
/var/folders/jv/r9wkk1k157jgllfhz9lygd0r0000gp/T/ipykernel_22416/157034202.py:31: SettingWithCopyWarning: 
A value is trying t

...

/var/folders/jv/r9wkk1k157jgllfhz9lygd0r0000gp/T/ipykernel_22416/157034202.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  target_codes_icd10['opcs'] = target_codes_icd10['concept_code_y']
/var/folders/jv/r9wkk1k157jgllfhz9lygd0r0000gp/T/ipykernel_22416/157034202.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  target_codes_icd9['opcs'] = target_codes_icd9['concept_code_y']
/var/folders/jv/r9wkk1k157jgllfhz9lygd0r0000gp/T/ipykernel_22416/157034202.py:31: SettingWithCopyWarning: 
A value is trying t

relationship_id
Subsumes    2993
Name: count, dtype: int64
relationship_id
Mapped from    13
Subsumes       10
Name: count, dtype: int64
relationship_id
Mapped from    17
Subsumes        4
Name: count, dtype: int64


/var/folders/jv/r9wkk1k157jgllfhz9lygd0r0000gp/T/ipykernel_22416/157034202.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  target_codes_icd10['opcs'] = target_codes_icd10['concept_code_y']
/var/folders/jv/r9wkk1k157jgllfhz9lygd0r0000gp/T/ipykernel_22416/157034202.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  target_codes_icd9['opcs'] = target_codes_icd9['concept_code_y']
/var/folders/jv/r9wkk1k157jgllfhz9lygd0r0000gp/T/ipykernel_22416/157034202.py:31: SettingWithCopyWarning: 
A value is trying t

relationship_id
Subsumes    3987
Name: count, dtype: int64
relationship_id
Mapped from    45
Subsumes       36
Name: count, dtype: int64
relationship_id
Mapped from    134
Subsumes        45
Name: count, dtype: int64
relationship_id
Subsumes    2993
Name: count, dtype: int64
relationship_id
Mapped from    13
Subsumes       10
Name: count, dtype: int64
relationship_id
Mapped from    17
Subsumes        4
Name: count, dtype: int64


/var/folders/jv/r9wkk1k157jgllfhz9lygd0r0000gp/T/ipykernel_22416/157034202.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  target_codes_icd10['opcs'] = target_codes_icd10['concept_code_y']
/var/folders/jv/r9wkk1k157jgllfhz9lygd0r0000gp/T/ipykernel_22416/157034202.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  target_codes_icd9['opcs'] = target_codes_icd9['concept_code_y']
/var/folders/jv/r9wkk1k157jgllfhz9lygd0r0000gp/T/ipykernel_22416/157034202.py:31: SettingWithCopyWarning: 
A value is trying t

In [212]:
omop_concept_relationships[omop_concept_relationships['concept_id_1']=='4096461']

,concept_id_1,concept_id_2,relationship_id,valid_start_date,valid_end_date,invalid_reason
624715,4096461,46274039,Value mapped from,20220801,20991231,NaN
11186518,4096461,40642537,Has status,20220128,20991231,NaN
11186519,4096461,40642539,Has Module,20220128,20991231,NaN
14623737,4096461,2002767,Mapped from,20180625,20991231,NaN
14623738,4096461,2002768,Subsumes,20180625,20991231,NaN
14623739,4096461,2002769,Subsumes,20180625,20991231,NaN
14623740,4096461,2002770,Subsumes,20180625,20991231,NaN
14623741,4096461,4018020,Concept was_a from,20140401,20991231,NaN
14623742,4096461,4018020,Mapped from,19700101,20991231,NaN
14623743,4096461,4018161,Concept was_a from,20140401,20991231,NaN


In [213]:
omop_concepts[omop_concepts['concept_id'] == '44814047']

,concept_id,concept_name,domain_id,vocabulary_id,concept_class_id,standard_concept,concept_code,valid_start_date,valid_end_date,invalid_reason
1428801,44814047,Sigmoid colectomy and end to side anastomosis,Procedure,SNOMED,Procedure,S,893461000000104,20140401,20991231,NaN
